In [23]:
from modules.data import *

ingest_zillow_csv("Neighborhood_zhvi_uc_condo_tier_0.33_0.67_sm_sa_mon.csv")


[Errno 2] No such file or directory: 'Neighborhood_zhvi_uc_condo_tier_0.33_0.67_sm_sa_mon.csv'


In [55]:
from modules.util import *

row1 = [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    37276,
    37460,
    37651,
    37952,
    38058,
    38147,
    38109,
    38101,
    38184,
    38647,
    39093,
    39520,
    39890,
    40411,
    40858,
    41117,
    41665,
    42387,
    43414,
    44525,
    45639,
    46056,
    46411,
    46702,
    47441,
    47927,
    48281,
    49026,
    49871,
    50543,
    51076,
    51435,
    51483,
    51701,
    52193,
    52892,
    52940,
    52706,
    52794,
    52991,
    53153,
    53272,
    53428,
    53594,
    54006,
    54499,
    54801,
    55224,
    55944,
    56879,
    57753,
    58391,
    58915,
    59221,
    59345,
    59918,
    60661,
    61593,
    62591,
    63113,
    63592,
    63920,
    64207,
    64466,
    64589,
    65087,
    65768,
    66390,
    66961,
    67279,
    67605,
    68135,
    68847,
    69765,
    70857,
    72287,
    73806,
    75221,
    76507,
    77530,
    78662,
    79761,
    81170,
    82412,
    83356,
    84237,
    85176,
    85970,
    86866,
    87708,
    88988,
    90375,
    91210,
    91957,
    92161,
    92813,
    93466,
    94187,
    95161,
    96678,
    98149,
    98918,
    98935,
    99229,
    100026,
    101035,
    101836,
    102872,
    104085,
    105594,
    106656,
    107366,
    108152,
    109696,
    111776,
    113775,
    115787,
    117832,
    120451,
    122818
  ]

header = [
    "RegionID",
    "SizeRank",
    "RegionName",
    "RegionType",
    "StateName",
    "State",
    "City",
    "Metro",
    "CountyName",
    "1996-01-31",
    "1996-02-29",
    "1996-03-31",
    "1996-04-30",
    "1996-05-31",
    "1996-06-30",
    "1996-07-31",
    "1996-08-31",
    "1996-09-30",
    "1996-10-31",
    "1996-11-30",
    "1996-12-31",
    "1997-01-31",
    "1997-02-28",
    "1997-03-31",
    "1997-04-30",
    "1997-05-31",
    "1997-06-30",
    "1997-07-31",
    "1997-08-31",
    "1997-09-30",
    "1997-10-31",
    "1997-11-30",
    "1997-12-31",
    "1998-01-31",
    "1998-02-28",
    "1998-03-31",
    "1998-04-30",
    "1998-05-31",
    "1998-06-30",
    "1998-07-31",
    "1998-08-31",
    "1998-09-30",
    "1998-10-31",
    "1998-11-30",
    "1998-12-31",
    "1999-01-31",
    "1999-02-28",
    "1999-03-31",
    "1999-04-30",
    "1999-05-31",
    "1999-06-30",
    "1999-07-31",
    "1999-08-31",
    "1999-09-30",
    "1999-10-31",
    "1999-11-30",
    "1999-12-31",
    "2000-01-31",
    "2000-02-29",
    "2000-03-31",
    "2000-04-30",
    "2000-05-31",
    "2000-06-30",
    "2000-07-31",
    "2000-08-31",
    "2000-09-30",
    "2000-10-31",
    "2000-11-30",
    "2000-12-31",
    "2001-01-31",
    "2001-02-28",
    "2001-03-31",
    "2001-04-30",
    "2001-05-31",
    "2001-06-30",
    "2001-07-31",
    "2001-08-31",
    "2001-09-30",
    "2001-10-31",
    "2001-11-30",
    "2001-12-31",
    "2002-01-31",
    "2002-02-28",
    "2002-03-31",
    "2002-04-30",
    "2002-05-31",
    "2002-06-30",
    "2002-07-31",
    "2002-08-31",
    "2002-09-30",
    "2002-10-31",
    "2002-11-30",
    "2002-12-31",
    "2003-01-31",
    "2003-02-28",
    "2003-03-31",
    "2003-04-30",
    "2003-05-31",
    "2003-06-30",
    "2003-07-31",
    "2003-08-31",
    "2003-09-30",
    "2003-10-31",
    "2003-11-30",
    "2003-12-31",
    "2004-01-31",
    "2004-02-29",
    "2004-03-31",
    "2004-04-30",
    "2004-05-31",
    "2004-06-30",
    "2004-07-31",
    "2004-08-31",
    "2004-09-30",
    "2004-10-31",
    "2004-11-30",
    "2004-12-31",
    "2005-01-31",
    "2005-02-28",
    "2005-03-31",
    "2005-04-30",
    "2005-05-31",
    "2005-06-30",
    "2005-07-31",
    "2005-08-31",
    "2005-09-30",
    "2005-10-31",
    "2005-11-30",
    "2005-12-31",
    "2006-01-31",
    "2006-02-28",
    "2006-03-31",
    "2006-04-30",
    "2006-05-31",
    "2006-06-30",
    "2006-07-31",
    "2006-08-31",
    "2006-09-30",
    "2006-10-31",
    "2006-11-30",
    "2006-12-31",
    "2007-01-31",
    "2007-02-28",
    "2007-03-31",
    "2007-04-30",
    "2007-05-31",
    "2007-06-30",
    "2007-07-31",
    "2007-08-31",
    "2007-09-30",
    "2007-10-31",
    "2007-11-30",
    "2007-12-31",
    "2008-01-31",
    "2008-02-29",
    "2008-03-31",
    "2008-04-30",
    "2008-05-31",
    "2008-06-30",
    "2008-07-31",
    "2008-08-31",
    "2008-09-30",
    "2008-10-31",
    "2008-11-30",
    "2008-12-31",
    "2009-01-31",
    "2009-02-28",
    "2009-03-31",
    "2009-04-30",
    "2009-05-31",
    "2009-06-30",
    "2009-07-31",
    "2009-08-31",
    "2009-09-30",
    "2009-10-31",
    "2009-11-30",
    "2009-12-31",
    "2010-01-31",
    "2010-02-28",
    "2010-03-31",
    "2010-04-30",
    "2010-05-31",
    "2010-06-30",
    "2010-07-31",
    "2010-08-31",
    "2010-09-30",
    "2010-10-31",
    "2010-11-30",
    "2010-12-31",
    "2011-01-31",
    "2011-02-28",
    "2011-03-31",
    "2011-04-30",
    "2011-05-31",
    "2011-06-30",
    "2011-07-31",
    "2011-08-31",
    "2011-09-30",
    "2011-10-31",
    "2011-11-30",
    "2011-12-31",
    "2012-01-31",
    "2012-02-29",
    "2012-03-31",
    "2012-04-30",
    "2012-05-31",
    "2012-06-30",
    "2012-07-31",
    "2012-08-31",
    "2012-09-30",
    "2012-10-31",
    "2012-11-30",
    "2012-12-31",
    "2013-01-31",
    "2013-02-28",
    "2013-03-31",
    "2013-04-30",
    "2013-05-31",
    "2013-06-30",
    "2013-07-31",
    "2013-08-31",
    "2013-09-30",
    "2013-10-31",
    "2013-11-30",
    "2013-12-31",
    "2014-01-31",
    "2014-02-28",
    "2014-03-31",
    "2014-04-30",
    "2014-05-31",
    "2014-06-30",
    "2014-07-31",
    "2014-08-31",
    "2014-09-30",
    "2014-10-31",
    "2014-11-30",
    "2014-12-31",
    "2015-01-31",
    "2015-02-28",
    "2015-03-31",
    "2015-04-30",
    "2015-05-31",
    "2015-06-30",
    "2015-07-31",
    "2015-08-31",
    "2015-09-30",
    "2015-10-31",
    "2015-11-30",
    "2015-12-31",
    "2016-01-31",
    "2016-02-29",
    "2016-03-31",
    "2016-04-30",
    "2016-05-31",
    "2016-06-30",
    "2016-07-31",
    "2016-08-31",
    "2016-09-30",
    "2016-10-31",
    "2016-11-30",
    "2016-12-31",
    "2017-01-31",
    "2017-02-28",
    "2017-03-31",
    "2017-04-30",
    "2017-05-31",
    "2017-06-30",
    "2017-07-31",
    "2017-08-31",
    "2017-09-30",
    "2017-10-31",
    "2017-11-30",
    "2017-12-31",
    "2018-01-31",
    "2018-02-28",
    "2018-03-31",
    "2018-04-30",
    "2018-05-31",
    "2018-06-30",
    "2018-07-31",
    "2018-08-31",
    "2018-09-30",
    "2018-10-31",
    "2018-11-30",
    "2018-12-31",
    "2019-01-31",
    "2019-02-28",
    "2019-03-31",
    "2019-04-30",
    "2019-05-31",
    "2019-06-30",
    "2019-07-31",
    "2019-08-31",
    "2019-09-30",
    "2019-10-31",
    "2019-11-30",
    "2019-12-31",
    "2020-01-31",
    "2020-02-29",
    "2020-03-31",
    "2020-04-30",
    "2020-05-31",
    "2020-06-30",
    "2020-07-31",
    "2020-08-31",
    "2020-09-30",
    "2020-10-31",
    "2020-11-30",
    "2020-12-31",
    "2021-01-31",
    "2021-02-28"
  ]

def inflate_zillow_row_by_date(row, header):
    """
    inflate_zillow_row_by_date inflates a single zillow row into multiple rows by date to create a long table from a wide table
    :param row: data from zillow row to be normalised
    :param header: row header columns
    :return: list of inflated rows
    """
    inflated_rows = []
    # +1 for addition of custom_column_housing_type in normalise_zillow_row, +1 for exclusive range and inclusive slice
    county_name_index = zillow_named_column_indexes.index(zillow_column_county_name) + 1
    common_row_metadata = row[:county_name_index]  # everything to housing type is common
    for index in range(county_name_index, len(header)):
        common_copy = copy.deepcopy(common_row_metadata)
        converted_header_to_timestamp = str(pd.to_datetime(header[index]))
        common_copy.append(converted_header_to_timestamp)  # add header date
        common_copy.append(row[index])  # housing value at date
        inflated_rows.append(common_copy)
    return inflated_rows

inflate_zillow_row_by_date(row1, header)


[[112345,
  1,
  'Maryvale',
  'Neighborhood',
  'AZ',
  'AZ',
  'Phoenix',
  'Phoenix-Mesa-Scottsdale',
  'Maricopa County',
  '1996-01-31 00:00:00',
  'Condo'],
 [112345,
  1,
  'Maryvale',
  'Neighborhood',
  'AZ',
  'AZ',
  'Phoenix',
  'Phoenix-Mesa-Scottsdale',
  'Maricopa County',
  '1996-02-29 00:00:00',
  0],
 [112345,
  1,
  'Maryvale',
  'Neighborhood',
  'AZ',
  'AZ',
  'Phoenix',
  'Phoenix-Mesa-Scottsdale',
  'Maricopa County',
  '1996-03-31 00:00:00',
  0],
 [112345,
  1,
  'Maryvale',
  'Neighborhood',
  'AZ',
  'AZ',
  'Phoenix',
  'Phoenix-Mesa-Scottsdale',
  'Maricopa County',
  '1996-04-30 00:00:00',
  0],
 [112345,
  1,
  'Maryvale',
  'Neighborhood',
  'AZ',
  'AZ',
  'Phoenix',
  'Phoenix-Mesa-Scottsdale',
  'Maricopa County',
  '1996-05-31 00:00:00',
  0],
 [112345,
  1,
  'Maryvale',
  'Neighborhood',
  'AZ',
  'AZ',
  'Phoenix',
  'Phoenix-Mesa-Scottsdale',
  'Maricopa County',
  '1996-06-30 00:00:00',
  0],
 [112345,
  1,
  'Maryvale',
  'Neighborhood',
  'AZ

In [56]:
data = [
  [
    "RegionID",
    "SizeRank",
    "RegionName",
    "RegionType",
    "StateName",
    "State",
    "City",
    "Metro",
    "CountyName",
    "Housing Type",
    "Date",
    "ZHVI"
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-02-29 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1996-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1997-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1998-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "1999-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-02-29 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2000-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2001-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2002-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2003-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-02-29 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2004-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2005-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2006-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2007-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-02-29 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2008-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2009-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-03-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-04-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-05-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-06-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-07-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-08-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-09-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-10-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-11-30 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2010-12-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-01-31 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-02-28 00:00:00",
    0
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-03-31 00:00:00",
    37276
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-04-30 00:00:00",
    37460
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-05-31 00:00:00",
    37651
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-06-30 00:00:00",
    37952
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-07-31 00:00:00",
    38058
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-08-31 00:00:00",
    38147
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-09-30 00:00:00",
    38109
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-10-31 00:00:00",
    38101
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-11-30 00:00:00",
    38184
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2011-12-31 00:00:00",
    38647
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-01-31 00:00:00",
    39093
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-02-29 00:00:00",
    39520
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-03-31 00:00:00",
    39890
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-04-30 00:00:00",
    40411
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-05-31 00:00:00",
    40858
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-06-30 00:00:00",
    41117
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-07-31 00:00:00",
    41665
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-08-31 00:00:00",
    42387
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-09-30 00:00:00",
    43414
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-10-31 00:00:00",
    44525
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-11-30 00:00:00",
    45639
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2012-12-31 00:00:00",
    46056
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-01-31 00:00:00",
    46411
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-02-28 00:00:00",
    46702
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-03-31 00:00:00",
    47441
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-04-30 00:00:00",
    47927
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-05-31 00:00:00",
    48281
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-06-30 00:00:00",
    49026
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-07-31 00:00:00",
    49871
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-08-31 00:00:00",
    50543
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-09-30 00:00:00",
    51076
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-10-31 00:00:00",
    51435
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-11-30 00:00:00",
    51483
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2013-12-31 00:00:00",
    51701
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-01-31 00:00:00",
    52193
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-02-28 00:00:00",
    52892
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-03-31 00:00:00",
    52940
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-04-30 00:00:00",
    52706
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-05-31 00:00:00",
    52794
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-06-30 00:00:00",
    52991
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-07-31 00:00:00",
    53153
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-08-31 00:00:00",
    53272
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-09-30 00:00:00",
    53428
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-10-31 00:00:00",
    53594
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-11-30 00:00:00",
    54006
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2014-12-31 00:00:00",
    54499
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-01-31 00:00:00",
    54801
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-02-28 00:00:00",
    55224
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-03-31 00:00:00",
    55944
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-04-30 00:00:00",
    56879
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-05-31 00:00:00",
    57753
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-06-30 00:00:00",
    58391
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-07-31 00:00:00",
    58915
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-08-31 00:00:00",
    59221
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-09-30 00:00:00",
    59345
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-10-31 00:00:00",
    59918
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-11-30 00:00:00",
    60661
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2015-12-31 00:00:00",
    61593
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-01-31 00:00:00",
    62591
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-02-29 00:00:00",
    63113
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-03-31 00:00:00",
    63592
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-04-30 00:00:00",
    63920
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-05-31 00:00:00",
    64207
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-06-30 00:00:00",
    64466
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-07-31 00:00:00",
    64589
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-08-31 00:00:00",
    65087
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-09-30 00:00:00",
    65768
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-10-31 00:00:00",
    66390
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-11-30 00:00:00",
    66961
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2016-12-31 00:00:00",
    67279
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-01-31 00:00:00",
    67605
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-02-28 00:00:00",
    68135
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-03-31 00:00:00",
    68847
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-04-30 00:00:00",
    69765
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-05-31 00:00:00",
    70857
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-06-30 00:00:00",
    72287
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-07-31 00:00:00",
    73806
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-08-31 00:00:00",
    75221
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-09-30 00:00:00",
    76507
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-10-31 00:00:00",
    77530
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-11-30 00:00:00",
    78662
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2017-12-31 00:00:00",
    79761
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-01-31 00:00:00",
    81170
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-02-28 00:00:00",
    82412
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-03-31 00:00:00",
    83356
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-04-30 00:00:00",
    84237
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-05-31 00:00:00",
    85176
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-06-30 00:00:00",
    85970
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-07-31 00:00:00",
    86866
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-08-31 00:00:00",
    87708
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-09-30 00:00:00",
    88988
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-10-31 00:00:00",
    90375
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-11-30 00:00:00",
    91210
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2018-12-31 00:00:00",
    91957
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-01-31 00:00:00",
    92161
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-02-28 00:00:00",
    92813
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-03-31 00:00:00",
    93466
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-04-30 00:00:00",
    94187
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-05-31 00:00:00",
    95161
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-06-30 00:00:00",
    96678
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-07-31 00:00:00",
    98149
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-08-31 00:00:00",
    98918
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-09-30 00:00:00",
    98935
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-10-31 00:00:00",
    99229
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-11-30 00:00:00",
    100026
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2019-12-31 00:00:00",
    101035
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-01-31 00:00:00",
    101836
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-02-29 00:00:00",
    102872
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-03-31 00:00:00",
    104085
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-04-30 00:00:00",
    105594
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-05-31 00:00:00",
    106656
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-06-30 00:00:00",
    107366
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-07-31 00:00:00",
    108152
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-08-31 00:00:00",
    109696
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-09-30 00:00:00",
    111776
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-10-31 00:00:00",
    113775
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-11-30 00:00:00",
    115787
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2020-12-31 00:00:00",
    117832
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2021-01-31 00:00:00",
    120451
  ],
  [
    112345,
    1,
    "Maryvale",
    "Neighborhood",
    "AZ",
    "AZ",
    "Phoenix",
    "Phoenix-Mesa-Scottsdale",
    "Maricopa County",
    "Condo",
    "2021-02-28 00:00:00",
    122818
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-01-31 00:00:00",
    80882
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-02-29 00:00:00",
    80697
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-03-31 00:00:00",
    80649
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-04-30 00:00:00",
    80461
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-05-31 00:00:00",
    80406
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-06-30 00:00:00",
    80400
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-07-31 00:00:00",
    80434
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-08-31 00:00:00",
    80604
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-09-30 00:00:00",
    80800
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-10-31 00:00:00",
    81010
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-11-30 00:00:00",
    81149
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1996-12-31 00:00:00",
    81297
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-01-31 00:00:00",
    81476
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-02-28 00:00:00",
    81769
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-03-31 00:00:00",
    81905
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-04-30 00:00:00",
    82259
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-05-31 00:00:00",
    82422
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-06-30 00:00:00",
    82513
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-07-31 00:00:00",
    82494
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-08-31 00:00:00",
    82382
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-09-30 00:00:00",
    82438
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-10-31 00:00:00",
    82451
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-11-30 00:00:00",
    82574
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1997-12-31 00:00:00",
    82796
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-01-31 00:00:00",
    83185
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-02-28 00:00:00",
    83593
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-03-31 00:00:00",
    83978
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-04-30 00:00:00",
    83995
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-05-31 00:00:00",
    84047
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-06-30 00:00:00",
    84195
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-07-31 00:00:00",
    84417
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-08-31 00:00:00",
    84580
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-09-30 00:00:00",
    84451
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-10-31 00:00:00",
    84512
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-11-30 00:00:00",
    84562
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1998-12-31 00:00:00",
    84455
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-01-31 00:00:00",
    83993
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-02-28 00:00:00",
    83515
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-03-31 00:00:00",
    83361
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-04-30 00:00:00",
    83471
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-05-31 00:00:00",
    83705
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-06-30 00:00:00",
    83861
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-07-31 00:00:00",
    84210
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-08-31 00:00:00",
    84432
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-09-30 00:00:00",
    84652
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-10-31 00:00:00",
    84619
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-11-30 00:00:00",
    84710
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "1999-12-31 00:00:00",
    84889
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-01-31 00:00:00",
    85175
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-02-29 00:00:00",
    85332
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-03-31 00:00:00",
    85430
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-04-30 00:00:00",
    85541
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-05-31 00:00:00",
    85592
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-06-30 00:00:00",
    85687
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-07-31 00:00:00",
    85637
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-08-31 00:00:00",
    85719
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-09-30 00:00:00",
    86043
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-10-31 00:00:00",
    86505
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-11-30 00:00:00",
    87040
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2000-12-31 00:00:00",
    87371
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-01-31 00:00:00",
    87794
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-02-28 00:00:00",
    88311
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-03-31 00:00:00",
    88719
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-04-30 00:00:00",
    89040
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-05-31 00:00:00",
    89388
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-06-30 00:00:00",
    89923
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-07-31 00:00:00",
    90479
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-08-31 00:00:00",
    91076
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-09-30 00:00:00",
    91501
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-10-31 00:00:00",
    91933
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-11-30 00:00:00",
    92147
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2001-12-31 00:00:00",
    92539
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-01-31 00:00:00",
    92831
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-02-28 00:00:00",
    93160
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-03-31 00:00:00",
    93537
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-04-30 00:00:00",
    94005
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-05-31 00:00:00",
    94356
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-06-30 00:00:00",
    94484
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-07-31 00:00:00",
    94600
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-08-31 00:00:00",
    94811
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-09-30 00:00:00",
    95067
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-10-31 00:00:00",
    95470
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-11-30 00:00:00",
    95950
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2002-12-31 00:00:00",
    96547
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-01-31 00:00:00",
    97081
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-02-28 00:00:00",
    97643
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-03-31 00:00:00",
    98027
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-04-30 00:00:00",
    98419
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-05-31 00:00:00",
    98771
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-06-30 00:00:00",
    99234
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-07-31 00:00:00",
    99804
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-08-31 00:00:00",
    100581
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-09-30 00:00:00",
    101801
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-10-31 00:00:00",
    103205
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-11-30 00:00:00",
    104807
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2003-12-31 00:00:00",
    106082
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-01-31 00:00:00",
    107804
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-02-29 00:00:00",
    109365
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-03-31 00:00:00",
    111850
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-04-30 00:00:00",
    113831
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-05-31 00:00:00",
    117050
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-06-30 00:00:00",
    120341
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-07-31 00:00:00",
    124170
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-08-31 00:00:00",
    127165
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-09-30 00:00:00",
    129700
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-10-31 00:00:00",
    131869
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-11-30 00:00:00",
    134222
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2004-12-31 00:00:00",
    137370
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-01-31 00:00:00",
    140212
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-02-28 00:00:00",
    143823
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-03-31 00:00:00",
    146042
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-04-30 00:00:00",
    149521
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-05-31 00:00:00",
    151195
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-06-30 00:00:00",
    153389
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-07-31 00:00:00",
    154697
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-08-31 00:00:00",
    157004
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-09-30 00:00:00",
    159035
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-10-31 00:00:00",
    161003
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-11-30 00:00:00",
    162736
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2005-12-31 00:00:00",
    163915
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-01-31 00:00:00",
    165256
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-02-28 00:00:00",
    165826
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-03-31 00:00:00",
    167248
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-04-30 00:00:00",
    168299
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-05-31 00:00:00",
    169759
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-06-30 00:00:00",
    170509
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-07-31 00:00:00",
    171246
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-08-31 00:00:00",
    171349
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-09-30 00:00:00",
    171642
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-10-31 00:00:00",
    171490
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-11-30 00:00:00",
    171531
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2006-12-31 00:00:00",
    171554
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-01-31 00:00:00",
    171714
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-02-28 00:00:00",
    171956
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-03-31 00:00:00",
    171604
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-04-30 00:00:00",
    170042
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-05-31 00:00:00",
    168239
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-06-30 00:00:00",
    166585
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-07-31 00:00:00",
    165060
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-08-31 00:00:00",
    163546
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-09-30 00:00:00",
    161840
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-10-31 00:00:00",
    159791
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-11-30 00:00:00",
    156129
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2007-12-31 00:00:00",
    151869
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-01-31 00:00:00",
    147429
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-02-29 00:00:00",
    142980
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-03-31 00:00:00",
    138975
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-04-30 00:00:00",
    135770
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-05-31 00:00:00",
    133314
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-06-30 00:00:00",
    130263
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-07-31 00:00:00",
    126942
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-08-31 00:00:00",
    123330
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-09-30 00:00:00",
    119060
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-10-31 00:00:00",
    114949
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-11-30 00:00:00",
    111506
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2008-12-31 00:00:00",
    108972
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-01-31 00:00:00",
    106410
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-02-28 00:00:00",
    103748
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-03-31 00:00:00",
    100797
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-04-30 00:00:00",
    97658
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-05-31 00:00:00",
    94220
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-06-30 00:00:00",
    90805
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-07-31 00:00:00",
    87552
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-08-31 00:00:00",
    84685
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-09-30 00:00:00",
    82733
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-10-31 00:00:00",
    81129
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-11-30 00:00:00",
    79890
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2009-12-31 00:00:00",
    78428
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-01-31 00:00:00",
    76784
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-02-28 00:00:00",
    75363
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-03-31 00:00:00",
    73990
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-04-30 00:00:00",
    73195
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-05-31 00:00:00",
    72164
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-06-30 00:00:00",
    71696
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-07-31 00:00:00",
    70931
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-08-31 00:00:00",
    70410
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-09-30 00:00:00",
    69460
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-10-31 00:00:00",
    68818
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-11-30 00:00:00",
    68086
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2010-12-31 00:00:00",
    67169
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-01-31 00:00:00",
    65917
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-02-28 00:00:00",
    64681
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-03-31 00:00:00",
    63669
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-04-30 00:00:00",
    62347
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-05-31 00:00:00",
    61215
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-06-30 00:00:00",
    59711
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-07-31 00:00:00",
    58950
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-08-31 00:00:00",
    57906
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-09-30 00:00:00",
    57438
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-10-31 00:00:00",
    56825
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-11-30 00:00:00",
    56343
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2011-12-31 00:00:00",
    56071
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-01-31 00:00:00",
    56262
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-02-29 00:00:00",
    56446
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-03-31 00:00:00",
    56512
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-04-30 00:00:00",
    56836
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-05-31 00:00:00",
    57335
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-06-30 00:00:00",
    58224
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-07-31 00:00:00",
    58759
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-08-31 00:00:00",
    59746
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-09-30 00:00:00",
    60541
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-10-31 00:00:00",
    61414
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-11-30 00:00:00",
    62360
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2012-12-31 00:00:00",
    63489
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-01-31 00:00:00",
    64830
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-02-28 00:00:00",
    66206
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-03-31 00:00:00",
    68154
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-04-30 00:00:00",
    69800
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-05-31 00:00:00",
    71717
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-06-30 00:00:00",
    73488
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-07-31 00:00:00",
    75528
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-08-31 00:00:00",
    77283
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-09-30 00:00:00",
    79091
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-10-31 00:00:00",
    80581
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-11-30 00:00:00",
    81868
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2013-12-31 00:00:00",
    82479
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-01-31 00:00:00",
    83083
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-02-28 00:00:00",
    83692
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-03-31 00:00:00",
    84084
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-04-30 00:00:00",
    84406
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-05-31 00:00:00",
    84330
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-06-30 00:00:00",
    84222
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-07-31 00:00:00",
    84219
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-08-31 00:00:00",
    84097
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-09-30 00:00:00",
    83682
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-10-31 00:00:00",
    83805
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-11-30 00:00:00",
    84019
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2014-12-31 00:00:00",
    84924
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-01-31 00:00:00",
    85312
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-02-28 00:00:00",
    85919
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-03-31 00:00:00",
    86051
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-04-30 00:00:00",
    86215
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-05-31 00:00:00",
    86429
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-06-30 00:00:00",
    86678
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-07-31 00:00:00",
    86659
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-08-31 00:00:00",
    86929
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-09-30 00:00:00",
    87427
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-10-31 00:00:00",
    87810
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-11-30 00:00:00",
    87960
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2015-12-31 00:00:00",
    88419
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-01-31 00:00:00",
    89171
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-02-29 00:00:00",
    89740
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-03-31 00:00:00",
    90014
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-04-30 00:00:00",
    90560
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-05-31 00:00:00",
    91177
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-06-30 00:00:00",
    91835
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-07-31 00:00:00",
    92421
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-08-31 00:00:00",
    92945
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-09-30 00:00:00",
    93471
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-10-31 00:00:00",
    93468
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-11-30 00:00:00",
    93769
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2016-12-31 00:00:00",
    93359
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-01-31 00:00:00",
    93599
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-02-28 00:00:00",
    93842
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-03-31 00:00:00",
    95341
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-04-30 00:00:00",
    96730
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-05-31 00:00:00",
    98258
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-06-30 00:00:00",
    99789
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-07-31 00:00:00",
    101587
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-08-31 00:00:00",
    103157
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-09-30 00:00:00",
    105108
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-10-31 00:00:00",
    107744
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-11-30 00:00:00",
    110755
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2017-12-31 00:00:00",
    113744
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-01-31 00:00:00",
    115946
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-02-28 00:00:00",
    118373
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-03-31 00:00:00",
    119970
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-04-30 00:00:00",
    121615
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-05-31 00:00:00",
    123246
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-06-30 00:00:00",
    125801
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-07-31 00:00:00",
    128346
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-08-31 00:00:00",
    130967
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-09-30 00:00:00",
    132601
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-10-31 00:00:00",
    133695
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-11-30 00:00:00",
    133986
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2018-12-31 00:00:00",
    134787
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-01-31 00:00:00",
    135626
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-02-28 00:00:00",
    136522
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-03-31 00:00:00",
    136846
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-04-30 00:00:00",
    137269
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-05-31 00:00:00",
    138031
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-06-30 00:00:00",
    138674
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-07-31 00:00:00",
    138792
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-08-31 00:00:00",
    138242
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-09-30 00:00:00",
    137668
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-10-31 00:00:00",
    137716
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-11-30 00:00:00",
    138246
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2019-12-31 00:00:00",
    138715
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-01-31 00:00:00",
    139115
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-02-29 00:00:00",
    139244
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-03-31 00:00:00",
    139778
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-04-30 00:00:00",
    140388
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-05-31 00:00:00",
    140691
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-06-30 00:00:00",
    140578
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-07-31 00:00:00",
    140780
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-08-31 00:00:00",
    141524
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-09-30 00:00:00",
    142699
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-10-31 00:00:00",
    143514
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-11-30 00:00:00",
    144245
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2020-12-31 00:00:00",
    144799
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2021-01-31 00:00:00",
    145560
  ],
  [
    192689,
    2,
    "Paradise",
    "Neighborhood",
    "NV",
    "NV",
    "Las Vegas",
    "Las Vegas-Henderson-Paradise",
    "Clark County",
    "Condo",
    "2021-02-28 00:00:00",
    146462
  ]
]

In [65]:
import pandas as pd
import copy

copydata = copy.deepcopy(data)

columns = copydata.pop(0)
df = pd.DataFrame(copydata, columns=columns)
#df["datetime"] = pd.to_datetime(df["datetime"])
df.head()
#df2 = df.pivot(columns="Date", values="ZHVI")
#df2.head()

#df["Date"].min(), df["Date"].max()
#df.groupby([pd.to_datetime(df["Date"]), "RegionName"])["ZHVI"].mean()

Date        RegionName
1996-01-31  Maryvale           0
            Paradise       80882
1996-02-29  Maryvale           0
            Paradise       80697
1996-03-31  Maryvale           0
                           ...  
2020-12-31  Paradise      144799
2021-01-31  Maryvale      120451
            Paradise      145560
2021-02-28  Maryvale      122818
            Paradise      146462
Name: ZHVI, Length: 604, dtype: int64